In [5]:
original_features = ['mfcc_mean', 'mfcc_variance', 'mfcc_max', 'mfcc_min', 'mfcc_median', 'mfcc_25th_percentile', 'mfcc_75th_percentile',
                     'zcr_mean', 'zcr_variance', 'zcr_max', 'zcr_min', 'zcr_median', 'zcr_25th_percentile', 'zcr_75th_percentile',
                     'rms_mean', 'rms_variance', 'rms_max', 'rms_min', 'rms_median', 'rms_25th_percentile', 'rms_75th_percentile',
                     'pitch_mean', 'pitch_variance', 'pitch_max', 'pitch_min', 'pitch_median', 'pitch_25th_percentile', 'pitch_75th_percentile',
                     'mfcc_range', 'mfcc_skewness', 'mfcc_kurtosis', 'mfcc_energy']

In [6]:
from itertools import combinations

def generate_feature_combinations(features):
    n = len(features)
    all_combinations = []
    for r in range(n):
        for combo in combinations(range(n), r):
            selected_features = [features[i] for i in range(n) if i not in combo]
            all_combinations.append(selected_features)
    return all_combinations

In [7]:
# Sử dụng hàm
features = ['mfcc', 'zcr', 'rms', 'pitch']
features_combo = generate_feature_combinations(features)
print(features_combo)

statistics = ['mean', 'variance', 'max', 'min', 'median', '25th_percentile', '75th_percentile']
statistics_combo = generate_feature_combinations(statistics)
print(statistics_combo)

[['mfcc', 'zcr', 'rms', 'pitch'], ['zcr', 'rms', 'pitch'], ['mfcc', 'rms', 'pitch'], ['mfcc', 'zcr', 'pitch'], ['mfcc', 'zcr', 'rms'], ['rms', 'pitch'], ['zcr', 'pitch'], ['zcr', 'rms'], ['mfcc', 'pitch'], ['mfcc', 'rms'], ['mfcc', 'zcr'], ['pitch'], ['rms'], ['zcr'], ['mfcc']]
[['mean', 'variance', 'max', 'min', 'median', '25th_percentile', '75th_percentile'], ['variance', 'max', 'min', 'median', '25th_percentile', '75th_percentile'], ['mean', 'max', 'min', 'median', '25th_percentile', '75th_percentile'], ['mean', 'variance', 'min', 'median', '25th_percentile', '75th_percentile'], ['mean', 'variance', 'max', 'median', '25th_percentile', '75th_percentile'], ['mean', 'variance', 'max', 'min', '25th_percentile', '75th_percentile'], ['mean', 'variance', 'max', 'min', 'median', '75th_percentile'], ['mean', 'variance', 'max', 'min', 'median', '25th_percentile'], ['max', 'min', 'median', '25th_percentile', '75th_percentile'], ['variance', 'min', 'median', '25th_percentile', '75th_percentile'

In [8]:
from config.eng_config import Config

ModuleNotFoundError: No module named 'config.config'

In [ ]:
def get_n_mfcc_paths(n_mfcc):
    train_path = Config.n_mfcc_config[n_mfcc].train_path
    validation_path = Config.n_mfcc_config[n_mfcc].validation_path
    test_path = Config.n_mfcc_config[n_mfcc].test_path
    return train_path, validation_path, test_path

In [ ]:
import pandas as pd

In [ ]:
def load_data(file_path):
        data = pd.read_csv(file_path)
        X = data.drop(columns=['label', 'file_path'])
        y = data['label']
        return X, y

In [ ]:
train_path, validation_path, test_path = get_n_mfcc_paths(20)
X_train, y_train = load_data(train_path)
X_test, y_test = load_data(test_path)
X_val, y_val = load_data(validation_path)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
def test_models(X_train, X_val, y_train, y_val, scalers, models):
    best_accuracy = 0
    best_result = None

    for scaler in scalers:
        for model in models:
            # Scaling dữ liệu
            X_train_scaled = scaler.fit_transform(X_train)
            X_val_scaled = scaler.transform(X_val)

            # Huấn luyện mô hình
            model.fit(X_train_scaled, y_train)

            # Dự đoán trên tập validation
            y_val_pred = model.predict(X_val_scaled)

            # Tính toán độ chính xác và báo cáo phân loại
            val_accuracy = accuracy_score(y_val, y_val_pred)
            # Nếu độ chính xác tốt hơn kết quả tốt nhất trước đó, cập nhật kết quả tốt nhất
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_result = {
                    'scaler': type(scaler).__name__,
                    'model': type(model).__name__,
                    'accuracy': val_accuracy
                }
    return best_result

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

In [ ]:
scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
models = [RandomForestClassifier(random_state=42), ExtraTreesClassifier(random_state=42), SVC(kernel='linear', random_state=42)]

# Để lưu kết quả cho mỗi lần loại bỏ đặc trưng
results = []

# Khởi tạo các biến để lưu pipeline tốt nhất
best_accuracy = 0
best_pipeline = None

# Lặp qua mỗi đặc trưng và mỗi thống kê để tạo các tổ hợp
for feature in features_combo:
    for statistic in statistics_combo:
        combined_features = []
        for f in feature:
            for s in statistic:
                combined_features.append(f'{f}_{s}')
        
        features_to_use = []
        for f in combined_features:
            # Tạo danh sách các cột bắt đầu bằng feature
            cols_starting_with_feature = [col for col in X_train.columns if col.startswith(f)]
            features_to_use.extend(cols_starting_with_feature)

        # Tạo tập dữ liệu mới với các đặc trưng đã loại bỏ
        X_train_new = X_train[features_to_use]
        X_val_new = X_val[features_to_use]
        
        # Thử nghiệm mỗi tổ hợp scaler và mô hình
        result_dict = test_models(X_train_new, X_val_new, y_train, y_val, scalers, models)

        scaler = result_dict['scaler']
        model = result_dict['model']
        accuracy = result_dict['accuracy']
        results.append((feature, statistic, scaler, model, accuracy))
        
        
        # Cập nhật pipeline tốt nhất nếu có độ chính xác cao hơn
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_pipeline = (feature, statistic, scaler, model, accuracy)
            print(best_pipeline)

# In pipeline tốt nhất
print("Best Pipeline:")
print("Features:", best_pipeline[0])
print("Statistic:", best_pipeline[1])
print("Scaler:", best_pipeline[2])
print("Model:", best_pipeline[3])
print("Accuracy:", best_pipeline[4])